In [1]:
import os
import sys

sys.path.append('/root/diffuser_chain_hd')
# print(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
# exit()


import json
import numpy as np
from os.path import join
import pdb

from diffuser.guides.policies import Policy
import diffuser.datasets as datasets
import diffuser.utils as utils

import argparse


class Parser(utils.Parser):
    dataset: str = 'maze2d-large-v1'
    config: str = 'config.maze2d_390_actionWeight1'


#---------------------------------- setup ----------------------------------#
n_samples = 100

args = Parser().parse_args('plan', add_extras=False) # Discovered it later, so had to disable it as I am doing it here, until I test it


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xd . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

[ utils/setup ] Reading config: config.maze2d_390_actionWeight1:maze2d_large_v1
[ utils/setup ] Using overrides | config: config.maze2d_390_actionWeight1 | dataset: maze2d_large_v1
[ utils/setup ] Lazy fstring | diffusion_loadpath : f:diffuserdiffuserdiffusion_actW1/H{horizon}_T{n_diffusion_steps}_J{jump} --> diffuserdiffuserdiffusion_actW1/H384_T256_J1
[ utils/setup ] Setting exp_name to: plans_actW1/release_H384_T256_LimitsNormalizer_b1_condFalse_J1_rpdFalse


pybullet build time: Nov 28 2023 23:51:11


In [5]:
print(args.logbase, args.dataset, args.diffusion_loadpath, args.diffusion_epoch)

diffusion_experiment = utils.load_diffusion('/root/diffuser_chain_hd/logs/', args.dataset, args.diffusion_loadpath, epoch=args.diffusion_epoch)

diffusion = diffusion_experiment.ema
dataset = diffusion_experiment.dataset
renderer = diffusion_experiment.renderer

policy = Policy(diffusion, dataset.normalizer)


logs maze2d-large-v1 diffuserdiffuserdiffusion_actW1/H384_T256_J1 latest
[ utils/serialization ] Loaded config from /root/diffuser_chain_hd/logs/maze2d-large-v1/diffuserdiffuserdiffusion_actW1/H384_T256_J1/dataset_config.pkl

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    jump: 1
    jump_action: none
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/serialization ] Loaded config from /root/diffuser_chain_hd/logs/maze2d-large-v1/diffuserdiffuserdiffusion_actW1/H384_T256_J1/render_config.pkl

[utils/config ] Config: <class 'diffuser.utils.rendering.Maze2dRenderer'>
    env: maze2d-large-v1

[ utils/serialization ] Loaded config from /root/diffuser_chain_hd/logs/maze2d-large-v1/diffuserdiffuserdiffusion_actW1/H384_T256_J1/model_config.pkl

[utils/config ] Config: <class 'diffuser.models.temporal.TemporalUnet'>
    cond_dim: 4
    d

load datafile: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.42it/s]


[ utils/preprocessing ] Segmented maze2d-large-v1 | 1061 paths | min length: 67 | max length: 30470 | mean length: 3764.9057492931197
[ datasets/buffer ] Finalized replay buffer | 1062 episodes
[ datasets/buffer ] Fields:
    actions: (1062, 40000, 2)
    infos/goal: (1062, 40000, 2)
    infos/qpos: (1062, 40000, 2)
    infos/qvel: (1062, 40000, 2)
    observations: (1062, 40000, 4)
    rewards: (1062, 40000, 1)
    terminals: (1062, 40000, 1)
    timeouts: (1062, 40000, 1)
    next_observations: (1062, 40000, 4)
    normed_observations: (1062, 40000, 4)
    normed_actions: (1062, 40000, 2)
[ models/temporal ] Channel dimensions: [(4, 32), (32, 128), (128, 256)]
[(4, 32), (32, 128), (128, 256)]

[ utils/serialization ] Loading model epoch: 1960000

trainner load from /root/diffuser_chain_hd/logs/maze2d-large-v1/diffuserdiffuserdiffusion_actW1/H384_T256_J1/state_1960000.pt, step 1999000


In [6]:
diffusion

GaussianDiffusion(
  (model): TemporalUnet(
    (time_mlp): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=32, out_features=128, bias=True)
      (2): Mish()
      (3): Linear(in_features=128, out_features=32, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0): ResidualTemporalBlock(
          (blocks): ModuleList(
            (0): Conv1dBlock(
              (block): Sequential(
                (0): Conv1d(4, 32, kernel_size=(5,), stride=(1,), padding=(2,))
                (1): Rearrange('batch channels horizon -> batch channels 1 horizon')
                (2): GroupNorm(8, 32, eps=1e-05, affine=True)
                (3): Rearrange('batch channels 1 horizon -> batch channels horizon')
                (4): Mish()
              )
            )
            (1): Conv1dBlock(
              (block): Sequential(
                (0): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
                (1): Rearrange('batch channels hori